In [17]:
import numpy as np
import cv2
import glob
import os
import pandas as pd
from pathlib import Path

#### Preprocessing functionality
https://github.com/HzFu/EyeQ/tree/master

In [2]:
def prepare_img(img):
    img = cv2.resize(img, (0,0), fx = 0.5, fy = 0.5) # resize to half
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX) # normalize to standard range for 8-bit grayscale
    return img

def find_mask(img):
    """Finds the approximate of the mask in the image.
    Args:
        img (array): retina image in grayscale
    Returns:
        array: mask approximate
    """
    th = np.mean(img)/3-5
    _, mask = cv2.threshold(img, max(0,th), 1, cv2.THRESH_BINARY) # mask of 0s and 1s using computed threshold if positive
    nn_mask = np.zeros((mask.shape[0]+2,mask.shape[1]+2),np.uint8) # 2px larger mask to prevent floodFill bleeding
    new_mask = (1-mask).astype(np.uint8) # reverted mask
    _,new_mask,_,_ = cv2.floodFill(new_mask, nn_mask, (0,0), (0), cv2.FLOODFILL_MASK_ONLY)
    _,new_mask,_,_ = cv2.floodFill(new_mask, nn_mask, (new_mask.shape[1]-1,new_mask.shape[0]-1), (0), cv2.FLOODFILL_MASK_ONLY)
    mask = mask + new_mask # combine the two masks
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20,20))
    mask = cv2.erode(mask, kernel) # use erosion and dilation to remove noise
    mask = cv2.dilate(mask, kernel)
    return mask

def get_center(mask):
    """Find center coordinates of the circular region of retina.
    Args:
        mask (array): mask approximate
    Returns:
        array: x and y coordinates of the circle's center
    """
    center = np.array([0,0])
    x=mask.sum(axis=1)
    center[0]=np.where(x>x.max()*0.95)[0].mean().astype(int)
    x=mask.sum(axis=0)
    center[1]=np.where(x>x.max()*0.95)[0].mean().astype(int)
    return center

def get_radius(mask, center):
    """Find radius of the circular region of retina.
    Args:
        mask (array): mask approximate
        center (array): circle's center coordinates
    Returns:
        number: radius of the circular region of retina
    """
    mask=mask.astype(np.uint8)
    ksize=max(mask.shape[1]//400*2+1,3)
    kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(ksize,ksize))
    mask=cv2.morphologyEx(mask, cv2.MORPH_GRADIENT, kernel)
    index=np.where(mask>0)
    d_int=np.sqrt((index[0]-center[0])**2+(index[1]-center[1])**2)
    b_count=np.bincount(np.ceil(d_int).astype(int))
    radius=np.where(b_count>b_count.max()*0.995)[0].max()
    return int(radius * 2) # multiply by 2 to account for downsizing

def get_bbox(center, radius, img_shape):
    """Get bounding box of the circular region of retina.
    Args:
        center (array): center coordinates of the circular region of retina
        radius (number): radius of the circular region of retina
    Returns:
        tuple: coordinates of the bounding box top left corner, height, and width
    """
    s_h = max(0, int(center[0] - radius)) # y coord of the top left corner (ensure positive)
    s_w = max(0, int(center[1] - radius)) # x coord of the top left corner (ensure positive)
    # bbox info containing: starting height, starting width, height, width
    return (s_h, s_w, min(img_shape[0]-s_h, 2*radius), min(img_shape[1]-s_w, 2*radius))

def get_mask(center, radius, img_shape):
    """Get mask of the circular region of retina.
    Args:
        center (array): center coordinates of the circular region of retina
        radius (number): radius of the circular region of retina
        img_shape (array): shape of the image
    Returns:
        array: mask of the circular region of retina
    """
    center_mask=np.zeros(shape=img_shape).astype('uint8')
    center = center[::-1]
    center_mask=cv2.circle(center_mask,center,radius,(1),-1)
    return center_mask

def apply_mask(img, mask):
    """Apply mask to retina image.
    Args:
        img (array): retina image in grayscale
        mask (array): mask of the circular region of retina
    Returns:
        array: retina image with mask applied
    """
    img[mask<=0,...]=0
    return img

def crop(img, bbox):
    """Crop retina image based on bounding box.
    Args:
        img (array): retina image
        bbox (tuple): coordinates of the bounding box top left corner, height, and width
    Returns:
        array: cropped retina image
    """
    top = bbox[0]
    bottom = bbox[0] + bbox[2]
    left = bbox[1]
    right = bbox[1] + bbox[3]
    img = img[top:bottom,left:right]
    return img

#### Preprocessing main

In [21]:
def get_dir(dir):
    os.path.normpath(dir)
    if not os.path.exists(dir):
        os.makedirs(dir)

def preprocess(in_path, out_path):
    unsuc_preprocess = []
    for img_path in glob.iglob(f'{in_path}/*.JPG'):
        # if Path(img_path).exists():
        #     continue
        img = cv2.imread(img_path) # BGR
        try:
            img_gs = prepare_img(img)
            approx_mask = find_mask(img_gs) # find mask approximation for calculations
            center = get_center(approx_mask)
            radius = get_radius(approx_mask, center)
            center = center * 2 # multiply by 2 to account for downsizing
            bbox = get_bbox(center, radius, img.shape[0:2])
            mask = get_mask(center, radius, img.shape[0:2])
            img = apply_mask(img, mask)
            img = crop(img, bbox)
        except:
            unsuc_preprocess.append(img_path)
            continue
        save_path = f'{out_path}/{img_path.split("/")[-1]}'
        cv2.imwrite(save_path, img)
    return unsuc_preprocess


Preprocess the StraDeSetB to later use for training.

In [ ]:
get_dir('./STRaDeSetB_preprocessed')
get_dir('./STRaDeSetB_preprocessed/Failure_cases')
unsuc_preprocess_B = []
unsuc_preprocess_B.append(preprocess('./STRaDeSetB', './STRaDeSetB_preprocessed'))
unsuc_preprocess_B.append(preprocess('./STRaDeSetB/Failure_cases', './STRaDeSetB_preprocessed/Failure_cases'))

In [23]:
def sobel_gradmap(img):
    grad_x = cv2.Sobel(img, cv2.CV_16S, 1, 0)
    grad_y = cv2.Sobel(img, cv2.CV_16S, 0, 1)
    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)
    grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
    return grad

def compute_features(img):
    rows,cols,_ = img.shape
    img_gs = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale
    O = sobel_gradmap(img_gs)
    img_lp_3 = cv2.blur(img_gs,(3,3))
    L1 = sobel_gradmap(img_lp_3)
    img_lp_5 = cv2.blur(img_gs,(5,5))
    L2 = sobel_gradmap(img_lp_5)
    # Compute focus measures
    FM1 = sum(O)/(rows*cols)
    FM2 = FM1 - sum(L1)/(rows*cols)
    FM3 = sum(L1)/(rows*cols) - sum(L2)/(rows*cols)
    return FM1, FM2, FM3

In [25]:
FM1_all = []
FM2_all = []
FM3_all = []
labels = []
for img_path in glob.iglob(f'./STRaDeSetB_preprocessed/*.JPG'):
    img = cv2.imread(img_path) # BGR
    FM1, FM2, FM3 = compute_features(img)
    FM1_all.append(FM1)
    FM2_all.append(FM2)
    FM3_all.append(FM3)
    labels.append('Usable')

for img_path in glob.iglob(f'./STRaDeSetB_preprocessed/Failure_cases/*.JPG'):
    img = cv2.imread(img_path) # BGR
    FM1, FM2, FM3 = compute_features(img)
    FM1_all.append(FM1)
    FM2_all.append(FM2)
    FM3_all.append(FM3)
    labels.append('Usable')
    
blur_data = pd.DataFrame({'FM1': FM1_all, 'FM2': FM2_all, 'FM3': FM3_all, 'label': labels})